# データサイエンス100本ノック 01~10

## はじめに

[100knocks-preprocess](https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess) からサンプルデータをダウンロードしてください。

- 初めに以下のセルを実行してください。
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います。
- pandas等、利用が想定されるライブラリは以下セルでインポートしています。
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません。
- 名前、住所等はダミーデータであり、実在するものではありません。

In [3]:
import polars as pl
from helper.polars import load_100knocks_data
pl.Config.set_fmt_str_lengths(100)
df_customer, df_category, df_product, df_receipt, df_store, df_geocode = load_100knocks_data()

## P-001

レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [4]:
df_receipt.head(5)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001""","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097""","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014""","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000""","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050""","""P060102007""",1,90


## P-002

レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [5]:
(
df_receipt
.select('sales_ymd', 'customer_id', 'product_cd', 'amount')
.head(5)
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90


## P-003

レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [6]:
(
df_receipt
.select(
    pl.col('sales_ymd').alias('sales_date'), 
    'customer_id', 
    'product_cd', 
    'amount')
.head(5)
)

sales_date,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90


In [7]:
(
df_receipt
.select('sales_ymd', 'customer_id', 'product_cd', 'amount')
.rename({'sales_ymd':'sales_date'})
.head(5)
)

sales_date,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90


## P-004

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"

In [8]:
(
df_receipt
.filter(pl.col.customer_id == 'CS018205000001')
.select('sales_ymd', 'customer_id', 'product_cd', 'amount')
.head(5)
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20180414,"""CS018205000001""","""P060104007""",600
20170614,"""CS018205000001""","""P050206001""",990
20170614,"""CS018205000001""","""P060702015""",108
20190216,"""CS018205000001""","""P071005024""",102


## P-005

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"
- 売上金額（amount）が1,000以上

In [9]:
(
df_receipt
.filter(
    (pl.col.customer_id == 'CS018205000001')
    & (pl.col.amount >= 1000)
)
.select('sales_ymd', 'customer_id', 'product_cd', 'amount')
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20190226,"""CS018205000001""","""P071401020""",2200
20180911,"""CS018205000001""","""P071401005""",1100


In [10]:
(
df_receipt
.filter(
    pl.col.customer_id
        .eq('CS018205000001')
        .and_(pl.col.amount.ge(1000))
)
.select('sales_ymd', 'customer_id', 'product_cd', 'amount')
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20190226,"""CS018205000001""","""P071401020""",2200
20180911,"""CS018205000001""","""P071401005""",1100


## P-006

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"
- 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [11]:
(
df_receipt
.filter(
    pl.col.customer_id
        .eq('CS018205000001')
        .and_(
            pl.col.amount
                .ge(1000)
                .or_(pl.col.quantity >= 5)
        )
)
.select('sales_ymd', 'customer_id', 'product_cd', 'amount', 'quantity')
)

sales_ymd,customer_id,product_cd,amount,quantity
i64,str,str,i64,i64
20180911,"""CS018205000001""","""P071401012""",2200,1
20180414,"""CS018205000001""","""P060104007""",600,6
20170614,"""CS018205000001""","""P050206001""",990,5
20190226,"""CS018205000001""","""P071401020""",2200,1
20180911,"""CS018205000001""","""P071401005""",1100,1


## P-007

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"
- 売上金額（amount）が1,000以上2,000以下

In [12]:
(
df_receipt
.filter(
    (pl.col.customer_id == 'CS018205000001')
    & pl.col.amount.is_between(1000, 2000)
)
.select('sales_ymd', 'customer_id', 'product_cd', 'amount')
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401005""",1100


## P-008

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"
- 商品コード（product_cd）が"P071401019"以外

In [13]:
(
df_receipt
.filter(
    (pl.col.customer_id == 'CS018205000001')
    & (pl.col.product_cd != 'P071401019')
)
.select('sales_ymd', 'customer_id', 'product_cd', 'amount')
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20180414,"""CS018205000001""","""P060104007""",600
20170614,"""CS018205000001""","""P050206001""",990
20170614,"""CS018205000001""","""P060702015""",108
20190216,"""CS018205000001""","""P071005024""",102
…,…,…,…
20190924,"""CS018205000001""","""P060805001""",495
20190226,"""CS018205000001""","""P071401020""",2200
20180911,"""CS018205000001""","""P071401005""",1100


## P-009

以下の処理において、出力結果を変えずにORをANDに書き換えよ。

```
df_store.query('not(prefecture_cd == "13" | floor_area > 900)')
```

In [14]:
(
df_store
.filter(
    (pl.col.prefecture_cd != '13')
    & (pl.col.floor_area <= 900)
)
)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f64,f64,f64
"""S14046""","""北山田店""","""14""","""神奈川県""","""神奈川県横浜市都筑区北山田一丁目""","""カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ""","""045-123-4049""",139.5916,35.56189,831.0
"""S14011""","""日吉本町店""","""14""","""神奈川県""","""神奈川県横浜市港北区日吉本町四丁目""","""カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ""","""045-123-4033""",139.6316,35.54655,890.0
"""S12013""","""習志野店""","""12""","""千葉県""","""千葉県習志野市芝園一丁目""","""チバケンナラシノシシバゾノイッチョウメ""","""047-123-4002""",140.022,35.66122,808.0


## P-010

店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [15]:
(
df_store
.filter(pl.col.store_cd.str.starts_with('S14'))
.head(5)
)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f64,f64,f64
"""S14010""","""菊名店""","""14""","""神奈川県""","""神奈川県横浜市港北区菊名一丁目""","""カナガワケンヨコハマシコウホククキクナイッチョウメ""","""045-123-4032""",139.6326,35.50049,1732.0
"""S14033""","""阿久和店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4043""",139.4961,35.45918,1495.0
"""S14036""","""相模原中央店""","""14""","""神奈川県""","""神奈川県相模原市中央二丁目""","""カナガワケンサガミハラシチュウオウニチョウメ""","""042-123-4045""",139.3716,35.57327,1679.0
"""S14040""","""長津田店""","""14""","""神奈川県""","""神奈川県横浜市緑区長津田みなみ台五丁目""","""カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ""","""045-123-4046""",139.4994,35.52398,1548.0
"""S14050""","""阿久和西店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4053""",139.4961,35.45918,1830.0
